In [1]:
import yt
import h5py
import pandas as pd

/mnt/bwpy/single/usr/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [23]:
ds = yt.load("RD0077/RedshiftOutput0077")

yt : [INFO     ] 2018-04-06 08:25:28,644 Parameters: current_time              = 18.251880636117
yt : [INFO     ] 2018-04-06 08:25:28,646 Parameters: domain_dimensions         = [512 512 512]
yt : [INFO     ] 2018-04-06 08:25:28,648 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2018-04-06 08:25:28,650 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2018-04-06 08:25:28,652 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2018-04-06 08:25:28,654 Parameters: current_redshift          = 11.599999246256
yt : [INFO     ] 2018-04-06 08:25:28,656 Parameters: omega_lambda              = 0.734
yt : [INFO     ] 2018-04-06 08:25:28,658 Parameters: omega_matter              = 0.266
yt : [INFO     ] 2018-04-06 08:25:28,659 Parameters: hubble_constant           = 0.71


In [25]:
ds.field_list

[('all', 'creation_time'),
 ('all', 'dynamical_time'),
 ('all', 'metallicity_fraction'),
 ('all', 'particle_index'),
 ('all', 'particle_mass'),
 ('all', 'particle_position_x'),
 ('all', 'particle_position_y'),
 ('all', 'particle_position_z'),
 ('all', 'particle_type'),
 ('all', 'particle_velocity_x'),
 ('all', 'particle_velocity_y'),
 ('all', 'particle_velocity_z'),
 ('enzo', 'Dark_Matter_Density'),
 ('enzo', 'Density'),
 ('enzo', 'Electron_Density'),
 ('enzo', 'GasEnergy'),
 ('enzo', 'H2II_Density'),
 ('enzo', 'H2I_Density'),
 ('enzo', 'H2I_kdiss'),
 ('enzo', 'HII_Density'),
 ('enzo', 'HI_Density'),
 ('enzo', 'HI_kph'),
 ('enzo', 'HM_Density'),
 ('enzo', 'HeIII_Density'),
 ('enzo', 'HeII_Density'),
 ('enzo', 'HeI_Density'),
 ('enzo', 'Metal_Density'),
 ('enzo', 'PhotoGamma'),
 ('enzo', 'RadAccel1'),
 ('enzo', 'RadAccel2'),
 ('enzo', 'RadAccel3'),
 ('enzo', 'SN_Colour'),
 ('enzo', 'Temperature'),
 ('enzo', 'TotalEnergy'),
 ('enzo', 'x-velocity'),
 ('enzo', 'y-velocity'),
 ('enzo', 'z-v

In [2]:
df = pd.read_csv('RD0077/sv.out', sep='\t', header=1, names=['# group', 'mass', '# part', 
                                                             'max-dens-x', 'max-dens-y', 'max-dens-z',
                                                             'center-of-mass', 'x', 'y', 'z',
                                                             'vx', 'vy', 'vz', 'max_r', 'rms-v'], index_col=False)

In [8]:
massive_halos = df[['mass', 'x', 'y', 'z', 'max_r']].head(10)
print(massive_halos)

           mass         x         y         z     max_r
0  5.350675e+09  0.446537  0.430071  0.551629  0.004200
1  1.462075e+09  0.443445  0.438602  0.454461  0.001891
2  1.198427e+09  0.433447  0.442860  0.466788  0.002190
3  1.052338e+09  0.489212  0.522910  0.525787  0.001611
4  1.046643e+09  0.441138  0.439269  0.527910  0.003553
5  9.299014e+08  0.444160  0.432626  0.552246  0.002340
6  9.058334e+08  0.442028  0.437707  0.542757  0.001750
7  7.653786e+08  0.440952  0.523491  0.447570  0.001808
8  7.443626e+08  0.435687  0.434774  0.564889  0.001450
9  7.362714e+08  0.511577  0.558761  0.532011  0.001531


In [53]:
fields = ["density", "temperature", "H_p1_fraction"]
f = h5py.File('halos.hdf5','w')

In [54]:
for row in massive_halos.iterrows():
    index, data = row
    x, y, z, r = data[['x', 'y', 'z', 'max_r']]
    sp = ds.sphere([x, y, z], (r, "unitary"))
    profile = sp.profile("radius", fields, weight_field="cell_mass")
    for field in fields:
        f.create_dataset("/Halo%02i/%s" % (index, field), data=profile[field])

In [55]:
f.close()